<a href="https://colab.research.google.com/github/NachoC20/RiesgoMercado/blob/main/RiesgoMercado/Tarea2/DNyDG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

# Tarea 2

In [48]:
%%R
#Instalamos los paquetes que necesitaremos.
install.packages("quantmod")
install.packages("data.table")
install.packages("PerformanceAnalytics")
install.packages("Deriv")

#Cargamos librerias
library(Deriv)
library(quantmod)
library(data.table)
library("PerformanceAnalytics")
library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.18.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 154512 bytes (150 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [49]:
%%R
## GENERAMOS LAS FUNCIONES QUE NECESITAREMOS:

# Función de interpolación de tasas por el método alamabrada
talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Función para diagonalizar un vector
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [50]:
%%R
#VaR con alisado

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

## **Definición de parámetros para valorar**

In [51]:
%%R
fval=as.Date("20211015",format="%Y%m%d") #Fecha de valoración

In [52]:
%%R
alpha = 0.98 #@param {type:"slider", min:0, max:1, step:0.01}

In [53]:
%%R
#poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
itpl = 0 #@param ["0", "1"] {type:"raw"} 

### Inciso 1
1000 acciones de GCarso, -5000 de AméricaMóvil y 1200 WalMart. *Todas de la BMV*

In [54]:
%%R
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#Claves de cotización de las emisoras, en orden alfabético
pos_eq=c(-5000,1000,1200) #Monto inicial invertido en acciones
nh=3660 #Días de historia

### Inciso 2
1500 dólares (peso dólar), 700 euros, y -600 libras esterlinas.

In [55]:
%%R
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #Simbolos en orden alfabetico
pos_fx=c(700,-600,1500) #Monto incial invertido en divisas
nh=3660 #días de historia

### Inciso 3
2 bonos largos, uno cupón cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 días, un bono M con tasa fija de 4.5% con un cupon de 4.05% anual vencimiento de 3600 días y un bono corto 1000 bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente). 

In [56]:
%%R

#CETES
base="RiesgosFinancieros/2022-1/Tarea/tasa_guber.txt"
ven_cete = cbind(180, 0)
contratos_cete = cbind(1500, 0)
nominal_cete = 10


#BONO M CON TASA FIJA
btasadesc="RiesgosFinancieros/2022-1/Tarea/tasa_yield.txt"
tf = cbind(0.045,0)
tfcupon = cbind(0.0405, 0)
ven_bonom = cbind(3600, 0)
plazocupon_bonom = cbind(182, 0)
contratos_bonom = cbind(1000, 0)
nominal_bonom = 100

#BONDES D
btasadescst="RiesgosFinancieros/2022-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2022-1/Tarea/tfondeo.txt"
plazos_bdm= cbind(707, 707) #Vencimiento del bono
plazocupon_bdm= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm = cbind(-1000, 0)
nominal_bdm=100

###Inciso 4
100 contratos de compra de futuros de peso dólar con un strike de 20.83 vencimiento de 5 días, y 50 contratos
de venta de futuros del IPC con strike de 49525 vencimiento de 53 días.

In [57]:
%%R
#FORWARDS TDC
bext="RiesgosFinancieros/2022-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2022-1/Tarea/tasa_fwd.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind(5,5)
contratos_fwd=cbind(100,0)
kst_fwd=cbind(20.83,20.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2022-1/Tarea/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53,53)
contratos_fwd_ind=cbind(50,0)
kst_fwd_ind=cbind(49525,49525)
nominal_fwd_ind=1

### Inciso 5

Un swap largo nocional de 1600 pagando tasa fija de 4.6% anual (cada 28 días) y recibiendo tasa flotante de la curva TIIE vencimiento de 588 días, un swap corto con nocional de 1200 pagando tasa variable de la TIIE y recibiendo fija de 4.9% vencimiento de 270.

In [58]:
%%R
btasadesc_sw="RiesgosFinancieros/2022-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2022-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.046,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(1600,-1200) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1, 1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

### Inciso 6
Dos opciones europeas, una de tasa de interés larga call con strike de 5.8 % vencimiento de 1700 días 1000
contratos, y otra put larga de tasa de interés con strike de 6.0 % de la tiie nocional de 500 contratos vencimiento
700 días. (se valuará basado en las superficies de volatilidad si es in o out the money y en las curvas de tasa de interés TIIE o Pagarés).

In [59]:
%%R
btasadesc_oir="RiesgosFinancieros/2022-1/Tarea/tasa_TIIE_SW_OP.txt" #Tasa de descuento
btasaspot_oir="RiesgosFinancieros/2022-1/Tarea/tasa_DIRS_SW_OP.txt" #Tasa Spot
bvolspot_oir="RiesgosFinancieros/2022-1/Tarea/tvoltiie_opc.txt" #Volatilidad
plazos_oir=cbind( 1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor) 
K_oir=cbind( 0.058,0.06) #Strike
contratos_oir=cbind(1000,500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple

## **Carga de datos**

### Inciso 1

In [60]:
%%R
start_date=Sys.Date()-nh #fecha inicial
dataEnv<-new.env()
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)

bt.prep(dataEnv,align='remove.na',fill.gaps = T)
stock_prices = dataEnv$prices

###Inciso 2

In [61]:
%%R
  start_date=Sys.Date()-nh #fecha inicial
  dataEnvFX<-new.env() #Creamos donde guardar los datos
  getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date) #Obtenemos los datps
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T) #Limpieza de datos
  stock_pricesFX = dataEnvFX$prices #Nos da el precio

  stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))


###Inciso 3

In [62]:
%%R

# CETES
data_cetes<-read.table(base)
n_gov<-nrow(data_cetes)
m_gov=ncol(data_cetes)
x_orig_gov=data.frame(data_cetes[2:n_gov,1:m_gov])
x_orig_gov=as.data.table(mutate(data_cetes[2:n_gov,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data_cetes[1,2:m_gov])

# BONO M
data_m<-read.table(btasadesc)
n_bonom<-nrow(data_m)
m_bonom=ncol(data_m)
x_orig_bonom=data.frame(data_m[2:n_bonom,1:m_bonom])
x_orig_bonom=as.data.table(mutate(data_m[2:n_bonom,1:m_bonom],Date=as.Date(V1,format="%Y%m%d")))
x_orig_bonom=x_orig_bonom%>%select(-V1)
nodos_bonom=data.frame(data_m[1,2:m_bonom])

# BONDE D

data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
X_orig_bd=as.data.table(mutate(data1[2:n,1:m_bd],Date=as.Date(V1,format="%Y%m%d")))
X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])

X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 

#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.


          V1         V2         V3         V4
2 2021-10-15 0.04850092 0.04790516 0.04729400
3 2021-10-14 0.04737299 0.04790516 0.04729400
4 2021-10-13 0.04737299 0.04790516 0.04751145
5 2021-10-12 0.04793244 0.04790516 0.04751145
6 2021-10-11 0.04737299 0.04790516 0.04751145
7 2021-10-08 0.04737299 0.04786294 0.04751145


###Inciso 4

In [63]:
%%R
#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))

x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)


###Para Dolar

if (yext==1)
{
#Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial


  
#Creación del objeto para guardar los datos
  dataEnvFX_ftdc<-new.env()
  
#obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX_ftdc,from=start_date, to=(fval))
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX_ftdc,align='remove.na',fill.gaps=T)
  
#muestra de datos
  head(dataEnvFX_ftdc$prices[,2])
  
#Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_ftdc$prices[,2])),coredata(dataEnvFX_ftdc$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

###Inciso 5

In [64]:
%%R
data1<-read.table(btasadesc_sw)
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])

###Inciso 6

In [65]:
%%R
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])

##**Integración de Insumos**

### Cruce de tablas

In [66]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  

lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield

#lin_gub_aux=data.table(Date=as.Date(lin_gub[x_orig_bonom,on=.(Date),nomatch=0]$Date)) 

lin_gub_bmybdst_aux=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)


lin_gub_bmybdst=data.table(Date=as.Date(X3_ftdc[lin_gub_bmybdst_aux,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)

lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield,  st (bonde), libor

lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd

lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index



lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

           Date
  1: 2021-10-14
  2: 2021-10-13
  3: 2021-10-12
  4: 2021-10-11
  5: 2021-10-08
 ---           
249: 2020-10-20
250: 2020-10-19
251: 2020-10-16
252: 2020-10-15
253: 2020-10-14


### Formato de historia

#### Historia de todos

In [67]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
n

[1] 253


#### Incisos 1 y 2

In [68]:
%%R
#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#### Incisos 3

In [69]:
%%R
#Historia de curva gubernamental y yield
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#Historia de curvas de bonde
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
X2_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)



#### Incisos 4

In [70]:
%%R
#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#### Incisos 5

In [71]:
%%R
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#### Incisos 6

In [72]:
%%R
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

## Valuación y derivadas

### Acciones y divisas

In [73]:
%%R
#divisas y acciones CÁLCULO

x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
mu_acc_div=0
g_acc_div=cbind(t(pos_fx),t(pos_eq))
g_acc_divt=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
gamma_acc_divt=matrix(0,ncol(x0_acc_div),ncol(x0_acc_div))


m_fx=length(pos_fx)
m_acc=length(pos_eq)

### Swaps

In [84]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}


swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)


#derivada del swap con respecto aL tiempo
dswap_dtt=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*Xvp/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto a la tasa cupón variable
dswap_dtc=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  (((contratos_swT*plazocupon_swT)/360)/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto al valor presente
dswap_dvp=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

mgamma_swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  c11=matrix(0,1,ncol(contratos_swT))
  c21=-1*(((contratos_swT*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
  c12=-1*(((contratos_swT*plazocupon_swT*VTplazos_sw)/360^2)/(1+Xvp*VTplazos_sw/360)^2)*nominal_swT*(-1)^por_swT
  c22=2*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw^2/360^3)))/(1+Xvp*VTplazos_sw/360)^(3))*nominal_swT*(-1)^por_swT
  rbind(cbind(diagv(c11),diagv(c12)),cbind(diagv(c21),diagv(c22)))
}

X_sw=cbind( XtfwdT,Xvp)

mu_sw=sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))
g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw

gamma_sw=mgamma_swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,])
tgamma_sw=c( XtfwdT[1,],Xvp[1,])%*%t(c( XtfwdT[1,],Xvp[1,]))*gamma_sw 

## Medicion de riesgo, Delta Gama

### Swaps

#### Total

In [88]:
%%R
DeltaXvp=Xvp[1:(n-1),]/Xvp[2:n,]-1
DeltaXtfwdt=XtfwdT[1:(n-1),]/XtfwdT[2:n,]-1

DeltaX_sw=cbind(DeltaXtfwdt,DeltaXvp)

VarCovX_sw=cov(DeltaX_sw)*((n-1)/(n-2))  #Matriz de Varianza-Covarianza


# VaR y CVaR Delta Gamma Normal Total
tgammavar_sw=(tgamma_sw%*%VarCovX_sw)
tgammavar2_sw=(tgamma_sw%*%VarCovX_sw)%*%(tgamma_sw%*%VarCovX_sw) 
VaRTotalDGM_sw=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))-(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*qnorm(alpha) #VaR Normal
CVaRTotalDGM_sw=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))-(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*dnorm(qnorm(alpha))/(1-alpha)

# VaR y CVaR Delta Gamma CF Total
tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw%*%VarCovX_sw)
asimPmargc_sw=sum(diag(tgammavar3_sw))+3*gt_sw%*%VarCovX_sw%*%tgamma_sw%*%VarCovX_sw%*%t(gt_sw) #gamma tercer momento
asimPmargc_sw=asimPmargc_sw/(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))^3  #coef de asimetría
VaRmargcstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
VaRTotalCF_sw=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))+(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*VaRmargcstd_sw #VaR cf
CVaRTotalCF_sw=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))-(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*dnorm(VaRmargcstd_sw)/(pnorm(VaRmargcstd_sw))

print("VaR Total Delta Gamma Normal")
print(VaRTotalDGM_sw)
print("CVaR Total Delta Gamma Normal")
print(CVaRTotalDGM_sw)
print("VaR Total Delta Gamma CF")
print(VaRTotalCF_sw)
print("CVaR Total Delta Gamma CF")
print(CVaRTotalCF_sw)



[1] "VaR Total Delta Gamma Normal"
             [,1]
[1,] -0.002164163
[1] "CVaR Total Delta Gamma Normal"
             [,1]
[1,] -0.002387909
[1] "VaR Total Delta Gamma CF"
            [,1]
[1,] -0.00216391
[1] "CVaR Total Delta Gamma CF"
             [,1]
[1,] -0.002387685


#### Por contrato

In [89]:
%%R
VaRcont_sw=matrix(0,1,ncol(N))
CVaRcont_sw=matrix(0,1,ncol(N))
VaRContDGM_sw=matrix(0,1,ncol(N))
CVaRContDGM_sw=matrix(0,1,ncol(N))
VaRContstd_sw=matrix(0,1,ncol(N))
VaRContCF_sw=matrix(0,1,ncol(N))
CVaRContCF_sw=matrix(0,1,ncol(N))
VaRmargc_sw=matrix(0,1,m)
VaRmargcstd_sw=matrix(0,1,m)
VaRmargcCF_sw=matrix(0,1,m)
CVaRmargc_sw=matrix(0,1,m)
CVaRmargcCF_sw=matrix(0,1,m)

VaRContDGM_swf=matrix(0,2,ncol(N))
CVaRContDGM_swf=matrix(0,2,ncol(N))
VaRContCF_swf=matrix(0,2,ncol(N))
CVaRContCF_swf=matrix(0,2,ncol(N))


#Valor de las derivadas evaluadas en el punto x0.
V0f_dtt_sw=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dtc_sw=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dvp_sw=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
for (j in (1:ncol(N)))
{
  if(j==1)
  {
    x0cont_swf=rbind(c( XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])]*0),
                     c( XtfwdT[1,1:sum(N[1:j])]*0,Xvp[1,1:sum(N[1:j])]))
    mucont_sw=sum(V0f_dtt_sw[1:sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc_sw[1:sum(N[1:j])],V0f_dvp_sw[1:sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])))
    VarCovXcont_sw=cov(cbind(XtfwdT[,1:sum(N[1:j])],Xvp[,1:sum(N[1:j])]))*(n/(n-1))	
    gamma_cont_sw=gamma_sw[1:(2*sum(N[1:j])),1:(2*sum(N[1:j]))]
    tgamma_sw= c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])%*%t(c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])]))*gamma_cont_sw #gamma tilde
  }
  else
  {
    x0cont_swf=rbind(c( XtfwdT[1,sum(N[1:j-1],1):sum(N[1:j])],Xvp[1,sum(N[1:j-1],1):sum(N[1:j])]*0),
                     c( XtfwdT[1,sum(N[1:j-1],1):sum(N[1:j])]*0,Xvp[1,sum(N[1:j-1],1):sum(N[1:j])]))
    mucont_sw=sum(V0f_dtt_sw[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc_sw[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp_sw[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])))
    VarCovXcont_sw=cov(cbind(XtfwdT[,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    gamma_cont_sw=gamma_sw[(2*sum(N[1:j-1])+1):(2*sum(N[1:j])),(2*sum(N[1:j-1])+1):(2*sum(N[1:j]))]
    tgamma_sw= c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])%*%t(c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]))*gamma_cont_sw #gamma tilde
  }
  
  
  tgammavar_sw=(tgamma_sw%*%VarCovXcont_sw)
  tgammavar2_sw=(tgamma_sw%*%VarCovXcont_sw)%*%(tgamma_sw%*%VarCovXcont_sw)
  VaRContDGM_sw[j]=(sum(diag(tgammavar_sw))/2+mucont_sw)-(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*qnorm(alpha) #VaR Normal
  CVaRContDGM_sw[j]=(sum(diag(tgammavar_sw))/2+mucont_sw)-(sqrt(sum(diag(tgammavar_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*dnorm(qnorm(alpha))/(1-alpha)
  ###VaR Cornish Fisher
  tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw%*%VarCovXcont_sw)
  asimPmargc_sw=sum(diag(tgammavar3_sw))+3*gtcont_sw%*%VarCovXcont_sw%*%tgamma_sw%*%VarCovXcont_sw%*%t(gtcont_sw) #gamma tercer momento
  asimPmargc_sw=asimPmargc_sw/(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))^3  #coef de asimetría
  VaRmargcstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
  VaRContCF_sw[j]=(sum(diag(tgammavar_sw))/2+mucont_sw)+(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*VaRmargcstd_sw #VaR Normal
  CVaRContCF_sw[j]=(sum(diag(tgammavar_sw))/2+sum(mucont_sw))-(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*dnorm(VaRmargcstd_sw)/(pnorm(VaRmargcstd_sw))
  
  #VaR Delta Normal
  VaRcont_sw[j]=sum(mucont_sw)-sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*qnorm(alpha)
  CVaRcont_sw[j]=sum(mucont_sw)-sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*dnorm(qnorm(alpha))/(1-alpha)

   for (k in (1:2))
  {
    gtcont_sw=t(as.matrix(gcont_sw*x0cont_swf[k,]))
    tgamma_sw= x0cont_swf[k,]%*%t(x0cont_swf[k,])*gamma_cont_sw #gamma tilde
    tgammavar_sw=(tgamma_sw%*%VarCovXcont_sw)
    tgammavar2_sw=(tgamma_sw%*%VarCovXcont_sw)%*%(tgamma_sw%*%VarCovXcont_sw)
    VaRContDGM_swf[k,j]=(sum(diag(tgammavar_sw))/2+mucont_sw)-(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*qnorm(alpha) #VaR Normal
    CVaRContDGM_swf[k,j]=(sum(diag(tgammavar_sw))/2+mucont_sw)-(sqrt(sum(diag(tgammavar_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*dnorm(qnorm(alpha))/(1-alpha)
    ###VaR Cornish Fisher
    tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw%*%VarCovXcont_sw)
    asimPmargc_sw=sum(diag(tgammavar3_sw))+3*gtcont_sw%*%VarCovXcont_sw%*%tgamma_sw%*%VarCovXcont_sw%*%t(gtcont_sw) #gamma tercer momento
    asimPmargc_sw=asimPmargc_sw/(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))^3  #coef de asimetría
    VaRmargcstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
    VaRContCF_swf[k,j]=(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*VaRmargcstd_sw #VaR Normal
    CVaRContCF_swf[k,j]=-(sqrt(sum(diag(tgammavar2_sw))/2+gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw)))*dnorm(VaRmargcstd_sw)/(pnorm(VaRmargcstd_sw))    
  }
}

print("VaR contrato Delta Gamma Normal")
print(VaRContDGM_sw) #VaR contrato Delta Gamma Normal 
print("CVaR contrato Delta Gamma Normal")
print(CVaRContDGM_sw) #CVaR contrato Delta Gamma Normal 
print("VaR contrato Delta Gamma CF")
print(VaRContCF_sw) #VaR contrato Delta Gamma CF
print("CVaR contrato Delta Gamma CF") 
print(CVaRContCF_sw) 
print("VaR contrato fr Delta Gamma Normal")
print(VaRContDGM_swf) #VaR contrato Delta Gamma Normal 
print("CVaR contrato fr Delta Gamma Normal")
print(CVaRContDGM_swf) #CVaR contrato Delta Gamma Normal 
print("VaR contrato fr Delta Gamma CF")
print(VaRContCF_swf) #VaR contrato Delta Gamma CF
print("CVaR contrato fr Delta Gamma CF") 
print(CVaRContCF_swf) 

#CVaR contrato Delta Gamma CF
#VaRContDGM_bdf #VaR contrato-factor de riesgo Delta Gamma Normal 
#CVaRContDGM_bdf #CVaR contrato-factor de riesgo Delta Gamma Normal 
#VaRContCF_bdf #VaR contrato-factor de riesgo Delta Gamma CF
#CVaRContCF_bdf


[1] "VaR contrato Delta Gamma Normal"
           [,1]        [,2]
[1,] -0.2291833 -0.05914348
[1] "CVaR contrato Delta Gamma Normal"
           [,1]        [,2]
[1,] -0.2700019 -0.06969723
[1] "VaR contrato Delta Gamma CF"
           [,1]        [,2]
[1,] -0.2291865 -0.05914348
[1] "CVaR contrato Delta Gamma CF"
           [,1]        [,2]
[1,] -0.2700145 -0.06969725
[1] "VaR contrato fr Delta Gamma Normal"
              [,1]          [,2]
[1,] -0.2293332817 -0.0591552915
[2,] -0.0009990088 -0.0001266701
[1] "CVaR contrato fr Delta Gamma Normal"
             [,1]          [,2]
[1,] -0.270188630 -0.0697111526
[2,] -0.001089237 -0.0001443528
[1] "VaR contrato fr Delta Gamma CF"
              [,1]          [,2]
[1,] -0.2285300970 -5.904580e-02
[2,] -0.0001958153 -1.717641e-05
[1] "CVaR contrato fr Delta Gamma CF"
              [,1]          [,2]
[1,] -0.2693854452 -6.960168e-02
[2,] -0.0002308261 -2.024724e-05


#### Por factor de riesgo

In [90]:
%%R
WFR_sw=rbind(c( XtfwdT[1,],Xvp[1,]*0),c( XtfwdT[1,]*0,Xvp[1,]))
VaRFr_sw=matrix(0,1,2)
CVaRFr_sw=matrix(0,1,2)
VaRFrDGN_sw=matrix(0,1,2)
VaRFrDGst_sw=matrix(0,1,2)
CVaRFrDGN_sw=matrix(0,1,2)
CVaRFrDGCF_sw=matrix(0,1,2)
VaRFrDGCF_sw=matrix(0,1,2)

for (j in (1:2))
{
   # gFr=as.matrix(V0f_dtc)
    #gtFr=as.matrix(gFr*XtfwdT[1,])
    #VarCovXFr=cov(cbind(XtfwdT[,]))*(n/(n-1))
    
    g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))
    
    #Multiplicación de la derivada por el punto x0
    gt_sw=WFR_sw[j,]*g_sw
    
    
    gamma_sw=mgamma_swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
    tgamma_sw=WFR_sw[j,]%*%t(WFR_sw[j,])*gamma_sw
    tgammavar_sw=(tgamma_sw%*%VarCovX_sw)
    tgammavar2_sw=(tgamma_sw%*%VarCovX_sw)%*%(tgamma_sw%*%VarCovX_sw) 
    VaRFrDGN_sw[j]=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))-(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*qnorm(alpha) #VaR Normal
    CVaRFrDGN_sw[j]=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))-(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*dnorm(qnorm(alpha))/(1-alpha)
    
    ###VaR Cornish Fisher
    tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw%*%VarCovX_sw)
    asimPmargc_sw=sum(diag(tgammavar3_sw))+3*gt_sw%*%VarCovX_sw%*%tgamma_sw%*%VarCovX_sw%*%t(gt_sw) #gamma tercer momento
    asimPmargc_sw=asimPmargc_sw/(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))^3  #coef de asimetría
    VaRFrDGst_sw[j]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
    VaRFrDGCF_sw[j]=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))+(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*VaRFrDGst_sw[j] #VaR Normal 
    CVaRFrDGCF_sw[j]=(sum(diag(tgammavar_sw))/2+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])))-(sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarCovX_sw%*%t(gt_sw)))*dnorm(VaRFrDGst_sw[j])/(pnorm(VaRFrDGst_sw[j]))

  #VaRFr[j]=-sqrt(gtFr%*%VarCovXFr%*%t(gtFr))*qnorm(alpha)
  #CVaRFr[j]=-sqrt(gtFr%*%VarCovXFr%*%t(gtFr))*dnorm(qnorm(alpha))/(1-alpha)
}
print("VaR factor de riesgo Delta Gamma Normal")
print(VaRFrDGN_sw)
print("CVaR factor de riesgo Delta Gamma Normal")
print(CVaRFrDGN_sw)
print("VaR factor de riesgo Delta Gamma CF")
print(VaRFrDGCF_sw)
print("CVaR factor de riesgo Delta Gamma CF")
print(CVaRFrDGCF_sw)


[1] "VaR factor de riesgo Delta Gamma Normal"
         [,1]         [,2]
[1,] -1.76041 -0.002164163
[1] "CVaR factor de riesgo Delta Gamma Normal"
          [,1]         [,2]
[1,] -2.074963 -0.002387909
[1] "VaR factor de riesgo Delta Gamma CF"
         [,1]        [,2]
[1,] -1.76041 -0.00216391
[1] "CVaR factor de riesgo Delta Gamma CF"
          [,1]         [,2]
[1,] -2.074963 -0.002387685


## Medicion de riesgo, Delta Normal

### Swaps

#### Por contrato

In [91]:
%%R
VaRcontDN_sw=matrix(0,1,ncol(N))
CVaRcontDN_sw=matrix(0,1,ncol(N))

V0f_dtt=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dtc=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dvp=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
for (j in (1:ncol(N)))
{
  if(j==1)
  {
    mucont_sw=sum(V0f_dtt[1:sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
  }
  else
  {
    mucont_sw=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    
  }
  
  
  
  #VaR Delta Normal
  VaRcontDN_sw[j]=sum(mucont_sw)+sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*qnorm(1-alpha)
  CVaRcontDN_sw[j]=sum(mucont_sw)+sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*dnorm(qnorm(1-alpha))/(1-alpha)
}

print("VaR por contrato Delta Normal")
print(VaRcontDN_sw)
print("CVaR por contrato Delta Normal")
print(CVaRcontDN_sw)

[1] "VaR por contrato Delta Normal"
          [,1]       [,2]
[1,] -1.436464 -0.4086602
[1] "CVaR por contrato Delta Normal"
         [,1]      [,2]
[1,] 1.691517 0.4814797


#### Por contrato factor de riesgo total

In [92]:
%%R
WFR=rbind(c( XtfwdT[1,],Xvp[1,]*0),c( XtfwdT[1,]*0,Xvp[1,]))
VaRFrDN_sw=matrix(0,1,2)
CVaRFrDN_sw=matrix(0,1,2)

for (j in (1:2))
{
    
    g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]),
               dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))
    
    #Multiplicación de la derivada por el punto x0
    gt_sw=WFR[j,]*g_sw
    VaRFrDN_sw[j]=sqrt(gt_sw%*%VarCovX_sw%*%t(gt_sw))*qnorm(1-alpha)
    CVaRFrDN_sw[j]=-sqrt(gt_sw%*%VarCovX_sw%*%t(gt_sw))*dnorm(qnorm(1-alpha))/(1-alpha)
}

print("VaR por contrato fr total")
print(VaRFrDN_sw)
print("CVaR por contrato fr total")
print(CVaRFrDN_sw)

[1] "VaR por contrato fr total"
          [,1]         [,2]
[1,] -1.759498 -0.001251557
[1] "CVaR por contrato fr total"
          [,1]         [,2]
[1,] -2.074051 -0.001475303


#### Contrato Factor de riesgo marginal

In [93]:
%%R
VaRcontfrDN_sw=matrix(0,2,ncol(N))
CVaRcontfrDN_sw=matrix(0,2,ncol(N))
WFRcont <- rbind(c(1,0), c(0,1))

V0f_dtt=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dtc=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dvp=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
for (j in (1:ncol(N)))
{
    for(k in (1:2)){
  if(j==1)
  {
    mucont_sw=sum(V0f_dtt[1:sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,1:sum(N[1:j])]*WFRcont[k,1],Xvp[1,1:sum(N[1:j])]*WFRcont[k,2])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
    }
  else
  {
    mucont_sw=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,1],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,2])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    }
  
  
  #VaR Delta Normal
  VaRcontfrDN_sw[k,j]=sum(mucont_sw)+sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*qnorm(1-alpha)
  CVaRcontfrDN_sw[k,j]=sum(mucont_sw)-sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*dnorm(qnorm(1-alpha))/(1-alpha)
}
}

print("VaR por contrato fr Delta Normal")
print(VaRcontfrDN_sw)
print("CVaR por contrato fr Delta Normal")
print(CVaRcontfrDN_sw)

[1] "VaR por contrato fr Delta Normal"
             [,1]          [,2]
[1,] -1.436451456 -0.4086563937
[2,] -0.001952618 -0.0002131249
[1] "CVaR por contrato fr Delta Normal"
             [,1]          [,2]
[1,] -1.693108722 -0.4816941513
[2,] -0.002158108 -0.0002316515


#### Total

In [97]:
%%R
#Evalución de las derivada en el punto x0

g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]),
        dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw
VaRTotalDN_sw=sqrt(gt_sw%*%VarCovX_sw%*%t(gt_sw))*qnorm(1-alpha)+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))
CVaRTotalDN_sw = -sqrt(gt_sw%*%VarCovX_sw%*%t(gt_sw))*dnorm(qnorm(1-alpha))/(1-alpha)+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))


print("VaR Total Delta Normal")
print(VaRTotalDN_sw)
print("CVaR Total Delta Normal")
print(CVaRTotalDN_sw)


[1] "VaR Total Delta Normal"
          [,1]
[1,] -1.760433
[1] "CVaR Total Delta Normal"
          [,1]
[1,] -2.074991
